In [1]:
import pandas as pd

books = pd.read_csv("books_classified.csv")
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories,predicted_category,predicted_category_score
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,A NOVEL THAT READERS and critics have been eag...,Fiction,Fiction,0.708725
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web:A Novel,A new 'Christie for Christmas' -- a full-lengt...,NaN,Fiction,0.575721
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"A memorable, mesmerizing heroine Jennifer -- b...",Fiction,Fiction,0.689205
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,Lewis' work on the nature of love divides love...,NaN,Non-fiction,0.580771
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"""In The Problem of Pain, C.S. Lewis, one of th...",NaN,Fiction,0.527924


In [2]:
import os
os.makedirs("D:/huggingface_cache", exist_ok=True)

os.environ["HF_HOME"] = "D:/huggingface_cache"
os.environ["TRANSFORMERS_CACHE"] = "D:/huggingface_cache"
os.environ["HF_DATASETS_CACHE"] = "D:/huggingface_cache"

print("HuggingFace cache directoty:", os.environ["HF_HOME"])

HuggingFace cache directoty: D:/huggingface_cache


In [3]:
from transformers import pipeline

sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

d:\book-recommender\.venv_Notebooks\Lib\site-packages\transformers\utils\hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Device set to use cpu


In [4]:
texts = books["tagged_description"].fillna("").astype(str).apply(lambda x: x[:512]).tolist()

sentiments = []
sentiment_scores = []

batch_size = 32
for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    results = sentiment_analyzer(batch)
    
    for res in results:
        sentiments.append(res["label"])
        sentiment_scores.append(res["score"])

In [5]:
books["sentiment"] = sentiments
books["sentiment_score"] = sentiment_scores

In [6]:
books.to_csv("books_sentiment.csv", index=False)